In [ ]:
# NaSch (Nagel–Schreckenberg) single-lane CA with ASCII animation
# Pure Python (standard library only): random, time, sys

import random, time, sys
from typing import List

# -----------------------
# Model parameters
# -----------------------
L = 120          # road length (cells)
V_MAX = 5        # maximum speed (cells per tick)
P_SLOW = 0.25    # random slowdown probability p
DENSITY = 0.12   # initial car density rho \in (0,1)
SEED = 7         # RNG seed for reproducibility

# Rendering
SLEEP = 0.05     # seconds between frames
SHOW_SPEEDS = True   # True: draw speeds 0..9; False: draw symbols ('·', '>', 'x')

random.seed(SEED)

# -----------------------
# Initialization
# -----------------------
# road[pos] = -1 if empty; otherwise integer speed in [0, V_MAX]
def init_ring(L: int, density: float, v_max: int) -> List[int]:
    n_cars = int(L * density)
    road = [-1] * L
    # choose random unique positions for cars
    positions = random.sample(range(L), n_cars)
    for pos in positions:
        road[pos] = random.randint(0, v_max)  # random initial speeds
    return road

def gap_ahead(road: List[int], i: int) -> int:
    """Number of empty cells in front of position i on a ring (0 if next cell occupied)."""
    L = len(road)
    g = 0
    k = (i + 1) % L
    while road[k] == -1 and g < L-1:
        g += 1
        k = (k + 1) % L
    return g

def step_ring(road: List[int], v_max: int, p_slow: float) -> List[int]:
    """One synchronous NaSch update on a ring."""
    L = len(road)
    speeds_new = [ -1 ] * L  # temporary store of new speeds (not positions)
    # 1) compute new speeds for all cars
    for i, v in enumerate(road):
        if v < 0:
            continue
        # Rule 1: accelerate
        v = min(v + 1, v_max)
        # Rule 2: slow down by gap to avoid collision
        g = gap_ahead(road, i)
        v = min(v, g)
        # Rule 3: randomization
        if v > 0 and random.random() < p_slow:
            v -= 1
        speeds_new[i] = v

    # 2) move all cars synchronously (Rule 4)
    new_road = [-1] * L
    for i, v in enumerate(speeds_new):
        if v < 0:
            continue
        j = (i + v) % L
        # If two cars try to move to same cell (shouldn’t happen with rules), keep the faster
        if new_road[j] < v:
            new_road[j] = v
    return new_road

# -----------------------
# ASCII rendering
# -----------------------
def render(road: List[int], show_speeds: bool = True) -> str:
    if show_speeds:
        # Draw speeds as digits; empty as '·'
        chars = [(str(v) if v >= 0 else '·') for v in road]
    else:
        # Visual shorthand: '>' if moving, 'x' if stopped, '·' if empty
        chars = [('>' if v > 0 else ('x' if v == 0 else '·')) for v in road]
    return ''.join(chars)

def clear_screen():
    # ANSI clear + home; works fine in Jupyter text output
    sys.stdout.write("\x1b[2J\x1b[H")
    sys.stdout.flush()

def animate(T=400, sleep=SLEEP, show_speeds=SHOW_SPEEDS):
    road = init_ring(L, DENSITY, V_MAX)
    moved_total = 0
    frames = 0
    clear_screen()
    for t in range(T):
        # draw
        sys.stdout.write(f"t={t:04d}  L={L}  vmax={V_MAX}  p={P_SLOW:.2f}  rho={DENSITY:.2f}\n")
        sys.stdout.write(render(road, show_speeds=show_speeds) + "\n")
        sys.stdout.write("legend: digit=speed, '·'=empty  |  or '>' moving, 'x' stopped\n")
        sys.stdout.flush()
        time.sleep(sleep)

        # one step and track average speed/flow
        before_positions = [i for i,v in enumerate(road) if v >= 0]
        road_next = step_ring(road, V_MAX, P_SLOW)
        after_positions = [i for i,v in enumerate(road_next) if v >= 0]
        # movement distance modulo ring; sum distances to estimate flow for display
        dist = 0
        Lloc = len(road)
        # map old positions to next by nearest moved car (approx, quick&dirty for display only)
        # better: sum of speeds in road_next equals total movement per tick
        dist = sum(v for v in road_next if v >= 0)
        moved_total += dist
        frames += 1

        road = road_next
        # move cursor up 2 lines to overwrite (cheap in-place animation)
        sys.stdout.write("\x1b[3A")

    # final stats line
    avg_flow = moved_total / frames / L  # cars-per-cell per tick (q)
    sys.stdout.write("\x1b[3B")  # move back down to bottom
    print(f"\nAverage per-tick movement sum: {moved_total/frames:.3f} cells/tick")
    print(f"Estimated flow q ≈ {avg_flow:.4f} cars-per-cell-per-tick")
    print("Done.")


In [ ]:
# Run the live ASCII animation
# Tip: Interrupt the kernel (stop button) to halt early.

animate(T=600, sleep=0.05, show_speeds=True)
